# Fine-Tuning T5 for SQL

In [1]:
import os
import torch
import time
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
from transformers import TextDataset, T5ForConditionalGeneration
from datasets import Dataset, DatasetDict, load_dataset, interleave_datasets, concatenate_datasets

2024-10-05 10:28:28.589609: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Environment
Check settings

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"        
print(f"Using device: {device}")

Using device: cpu


## Data
Load datasets

In [4]:
dataset_csql_train = load_dataset("b-mc2/sql-create-context", split='train[:8%]')
dataset_csql_test = load_dataset("b-mc2/sql-create-context", split='train[-2%:-1%]')
dataset_csql_val = load_dataset("b-mc2/sql-create-context", split='train[-1%:]')

dataset_tsql_train = load_dataset("Clinton/Text-to-sql-v1", split='train[:8%]')
dataset_tsql_train = dataset_tsql_train.remove_columns(['source', 'text'])
dataset_tsql_train = dataset_tsql_train.rename_columns({'instruction': 'question', 'input': 'context', 'response': 'answer'})

dataset_tsql_test  = load_dataset("Clinton/Text-to-sql-v1", split='train[-2%:-1%]')
dataset_tsql_test  = dataset_tsql_test.remove_columns(['source', 'text'])
dataset_tsql_test  = dataset_tsql_test.rename_columns({'instruction': 'question', 'input': 'context', 'response': 'answer'})

dataset_tsql_val   = load_dataset("Clinton/Text-to-sql-v1", split='train[-1%:]')
dataset_tsql_val   = dataset_tsql_val.remove_columns(['source', 'text'])
dataset_tsql_val   = dataset_tsql_val.rename_columns({'instruction': 'question', 'input': 'context', 'response': 'answer'})

In [5]:
dataset_train_merged = concatenate_datasets(
    [
        dataset_csql_train, 
        # dataset_tsql_train
        ]
    )
dataset_test_merged = concatenate_datasets(
    [
        dataset_csql_test, 
        # dataset_tsql_test
        ]
    )
dataset_val_merged = concatenate_datasets(
    [
        dataset_csql_val, 
        # dataset_tsql_val
        ]
    )

In [6]:
dataset_train_merged.to_csv('train_merged.csv', index=False)
dataset_test_merged.to_csv('test_merged.csv', index=False)
dataset_val_merged.to_csv('val_merged.csv', index=False)

Creating CSV from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

168122

In [7]:
dataset_train_merged

Dataset({
    features: ['context', 'question', 'answer'],
    num_rows: 6286
})

In [8]:
dataset_test_merged

Dataset({
    features: ['context', 'question', 'answer'],
    num_rows: 786
})

In [9]:
dataset_val_merged

Dataset({
    features: ['context', 'question', 'answer'],
    num_rows: 786
})

In [4]:
dataset = load_dataset('csv', data_files={
    "train": "train_merged.csv", 
    "test": "test_merged.csv", 
    "val": "val_merged.csv"
    })

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answer'],
        num_rows: 6286
    })
    test: Dataset({
        features: ['context', 'question', 'answer'],
        num_rows: 786
    })
    val: Dataset({
        features: ['context', 'question', 'answer'],
        num_rows: 786
    })
})

In [6]:
dataset['test'][0]

{'context': 'CREATE TABLE table_name_41 (entrant VARCHAR, year INTEGER)',
 'question': 'Who was the entrant before 1988?',
 'answer': 'SELECT entrant FROM table_name_41 WHERE year < 1988'}

## Model and Tokenizer
Define configuration settings

In [4]:
# model_name='google/flan-t5-small'
model_name='t5-small'
os.environ['TOKENIZERS_PARALLELISM'] = 'true' 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
base_model = base_model.to(device)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

Preprocess datasets

In [7]:
def tokenize_function(sample):
    start_prompt = "Tables:\n"
    middle_prompt = "\n\nQuestion:\n"
    end_prompt = "\n\nAnswer:\n"

    data_zip = zip(sample['context'], sample['question'])
    prompt = [start_prompt + context + middle_prompt + question + end_prompt for context, question in data_zip]
    sample['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    sample['labels'] = tokenizer(sample['answer'], padding="max_length", truncation=True, return_tensors="pt").input_ids
    return sample

In [8]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['question','context','answer'])


Map:   0%|          | 0/786 [00:00<?, ? examples/s]

In [9]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 6286
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 786
    })
    val: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 786
    })
})

In [11]:
tokenized_datasets.keys()

dict_keys(['train', 'test', 'val'])

In [12]:
tokenized_datasets['train'][0].keys()

dict_keys(['input_ids', 'labels'])

In [13]:
tokenized_datasets['train'][0]['input_ids'][:10]

[4398, 7, 10, 205, 4386, 6048, 332, 17098, 819, 41]

In [14]:
tokenized_datasets['train'][0]['labels'][:10]

[3, 23143, 14196, 2847, 17161, 599, 1935, 61, 21680, 819]

In [15]:
tokenized_datasets['train'].shape

(6286, 2)

In [16]:
tokenized_datasets['val'].shape

(786, 2)

In [17]:
tokenized_datasets['test'].shape

(786, 2)

Test model with zero shot prediction/inferencing

In [10]:
index = 0

question = dataset['test'][index]['question']
context = dataset['test'][index]['context']
answer = dataset['test'][index]['answer']

prompt = f"""Tables:
{context}

Question:
{question}

Answer:
"""

inputs = tokenizer(prompt, return_tensors='pt')
inputs = inputs.to('cpu')

output = tokenizer.decode(
    base_model.generate(
        inputs["input_ids"], 
        max_new_tokens=200,
    )[0], 
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN ANSWER:\n{answer}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Tables:
CREATE TABLE table_name_41 (entrant VARCHAR, year INTEGER)

Question:
Who was the entrant before 1988?

Answer:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN ANSWER:
SELECT entrant FROM table_name_41 WHERE year < 1988

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
Question: Who was the entrant before 1988? Answer: Who was the entrant before 1988?


Fine tune

In [19]:
try:
    finetuned_model = AutoModelForSeq2SeqLM.from_pretrained("finetuned_model_2_epoch")
    finetuned_model = finetuned_model.to('cpu')
    to_train = False

except:
    to_train = True
    finetuned_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    finetuned_model = finetuned_model.to('cpu')
    tokenizer = AutoTokenizer.from_pretrained(model_name)

In [20]:
%%time

if to_train:
    output_dir = 'training'

    training_args = TrainingArguments(
        output_dir=output_dir,
        learning_rate=5e-3,
        num_train_epochs=2,
        per_device_train_batch_size=16,     # batch size per device during training
        per_device_eval_batch_size=16,      # batch size for evaluation
        weight_decay=0.01,
        logging_steps=50,
        evaluation_strategy='steps',        # evaluation strategy to adopt during training
        eval_steps=500,  
        no_cuda=True,                 
    )

    trainer = Trainer(
        model=finetuned_model,
        args=training_args,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['val'],
    )
    
    trainer.train()
    
    finetuned_model.save_pretrained("model")

/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/transformers/training_args.py:1540: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


  0%|          | 0/786 [00:00<?, ?it/s]

{'loss': 0.332, 'grad_norm': 0.09357812255620956, 'learning_rate': 0.0046819338422391865, 'epoch': 0.13}
{'loss': 0.079, 'grad_norm': 0.08880062401294708, 'learning_rate': 0.004363867684478371, 'epoch': 0.25}
{'loss': 0.0532, 'grad_norm': 0.07429048418998718, 'learning_rate': 0.0040458015267175575, 'epoch': 0.38}
{'loss': 0.0422, 'grad_norm': 0.06269390881061554, 'learning_rate': 0.003727735368956743, 'epoch': 0.51}
{'loss': 0.0361, 'grad_norm': 0.04715965688228607, 'learning_rate': 0.003409669211195929, 'epoch': 0.64}
{'loss': 0.0322, 'grad_norm': 0.07090552151203156, 'learning_rate': 0.003091603053435115, 'epoch': 0.76}
{'loss': 0.031, 'grad_norm': 0.06011456251144409, 'learning_rate': 0.0027735368956743, 'epoch': 0.89}
{'loss': 0.0281, 'grad_norm': 0.03623080998659134, 'learning_rate': 0.002455470737913486, 'epoch': 1.02}
{'loss': 0.0232, 'grad_norm': 0.02512708678841591, 'learning_rate': 0.002137404580152672, 'epoch': 1.15}
{'loss': 0.0192, 'grad_norm': 0.0392158180475235, 'learnin

  0%|          | 0/50 [00:00<?, ?it/s]

{'eval_loss': 0.0701521635055542, 'eval_runtime': 275.8649, 'eval_samples_per_second': 2.849, 'eval_steps_per_second': 0.181, 'epoch': 1.27}
{'loss': 0.0211, 'grad_norm': 0.045559316873550415, 'learning_rate': 0.0015012722646310433, 'epoch': 1.4}
{'loss': 0.0203, 'grad_norm': 0.029621044173836708, 'learning_rate': 0.001183206106870229, 'epoch': 1.53}
{'loss': 0.0175, 'grad_norm': 0.03458360955119133, 'learning_rate': 0.0008651399491094148, 'epoch': 1.65}
{'loss': 0.0166, 'grad_norm': 0.03688393533229828, 'learning_rate': 0.0005470737913486006, 'epoch': 1.78}
{'loss': 0.0169, 'grad_norm': 0.0318748913705349, 'learning_rate': 0.00022900763358778625, 'epoch': 1.91}
{'train_runtime': 28249.7712, 'train_samples_per_second': 0.445, 'train_steps_per_second': 0.028, 'train_loss': 0.04972395581446835, 'epoch': 2.0}
CPU times: user 1d 5h 29min 5s, sys: 15h 4min 48s, total: 1d 20h 33min 53s
Wall time: 7h 50min 50s


Load model from local folder

In [11]:
model_path = "./model"
finetuned_model = T5ForConditionalGeneration.from_pretrained(model_path)

In [ ]:
# finetuned_model

Test fine-tuned model with zero shot inferencing

In [12]:
index = 1
# index = len(dataset['test'])-200

question = dataset['test'][index]['question']
context = dataset['test'][index]['context']
answer = dataset['test'][index]['answer']

prompt = f"""Tables:
{context}

Question:
{question}

Answer:
"""

inputs = tokenizer(prompt, return_tensors='pt')
inputs = inputs.to('cpu')

output = tokenizer.decode(
    finetuned_model.generate(
        inputs["input_ids"], 
        max_new_tokens=200,
    )[0], 
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN ANSWER:\n{answer}\n')
print(dash_line)
print(f'FINE-TUNED MODEL - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Tables:
CREATE TABLE table_name_89 (engine VARCHAR, year VARCHAR)

Question:
Which engine was used in 1987?

Answer:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN ANSWER:
SELECT engine FROM table_name_89 WHERE year = 1987

---------------------------------------------------------------------------------------------------
FINE-TUNED MODEL - ZERO SHOT:
SELECT engine FROM table_89 WHERE year = 1987


Test fine-tuned model with query

In [13]:
def get_sql(query):
    prompt = "translate English to SQL: %s " % query
    features = tokenizer([prompt], return_tensors='pt')
    output = finetuned_model.generate(
        input_ids=features['input_ids'],
        max_new_tokens=200
        )
    return tokenizer.decode(output[0], skip_special_tokens=True)
    

In [15]:
query = "Which engine was used in 1987?"
get_sql(query)

'SELECT engine FROM 1987'

In [16]:
for i in range(10,15, 1):
    print('Question: ' + dataset['test'][i]['question'])
    print('Predict. :' + get_sql(dataset['test'][i]['question']))
    print('Expected: ' + dataset['test'][i]['answer'])
    print('=================================\n')

Question: What is the division record for the Indians?
Predict. :SELECT division_ record FROM Indians
Expected: SELECT division_record FROM table_name_26 WHERE team = "indians"

Question: What is the overall record of Indian River?
Predict. :SELECT overall_registration FROM Indian River
Expected: SELECT overall_record FROM table_name_17 WHERE school = "indian river"

Question: What was the season outcome of Lake Forest?
Predict. :SELECT season FROM Lake Forest
Expected: SELECT season_outcome FROM table_name_43 WHERE school = "lake forest"

Question: How many Deciles are coed?
Predict. :SELECT CODEDEDELES COUNT(*) FROM Deciles
Expected: SELECT COUNT(decile) FROM table_name_51 WHERE gender = "coed"

Question: Which is the lowest Decile that is coed?
Predict. :SELECT MIN(Decle) FROM coed
Expected: SELECT MIN(decile) FROM table_name_94 WHERE gender = "coed"



Evaluate

In [21]:
import evaluate
import pandas as pd

In [18]:
rouge = evaluate.load('rouge')

In [24]:
questions = dataset['test'][0:10]['question']
answer = dataset['test'][0:10]['answer']

base_model_answers = []
finetuned_model_answers = []

for question in questions:
    prompt = f""" 
    Question:
    {question}

    Answer:
    """
    
    input_ids = tokenizer.encode(prompt, return_tensors='pt')

    finetuned_model_outputs = finetuned_model.generate(input_ids=input_ids,generation_config=GenerationConfig(max_new_tokens=200))
    finetuned_model_output = tokenizer.decode(finetuned_model_outputs[0], skip_special_tokens=True)


    base_model_outputs = base_model.generate(input_ids=input_ids,generation_config=GenerationConfig(max_new_tokens=200))
    base_model_output = tokenizer.decode(base_model_outputs[0], skip_special_tokens=True)

    finetuned_model_answers.append(finetuned_model_output)
    base_model_answers.append(base_model_output)

In [26]:
zipped_answers = list(zip(answer, finetuned_model_answers, base_model_answers))
df = pd.DataFrame(zipped_answers, columns=['human','finetuned','base'])
df

,human,finetuned,base
0,SELECT entrant FROM table_name_41 WHERE year <...,entrant entrant enrant enrant enrant enrant en...,Question: Who was the entrant before 1988? Ans...
1,SELECT engine FROM table_name_89 WHERE year = ...,SELECT engine__191987,Question: Which engine was used in 1987? Answe...
2,SELECT league FROM table_name_39 WHERE year = ...,SELECT league FROM league WATCHE19 2001,Question: What league did they play in 2001? A...
3,SELECT open_cup FROM table_name_21 WHERE reg_s...,"SELECT open cup FROM cup WHERE open cup = 2nd,...",Question: What open cup did they play in when ...
4,SELECT COUNT(week) FROM table_name_27 WHERE da...,"SELECT week FROM date of October 9, 1983, atte...",Question: What is the week with a date of Octo...
5,SELECT MIN(attendance) FROM table_name_30 WHER...,SELECT MIN(attendance) FROM week 14,Answer: What is the lowest attendance with wee...
6,SELECT game_site FROM table_name_90 WHERE week...,"SELECT Oakland Raiders, Oakland Raiders, Oakla...",Answer:
7,SELECT SUM(year) FROM table_name_43 WHERE entr...,SELECT COUNT(1986 years) FROM Cosworth v8 engine,Answer: How many years did Barclay Nordica Arr...
8,SELECT SUM(year) FROM table_name_98 WHERE engi...,SELECT COUNT(19 years) FROM bmw WHERE bmw str-...,True
9,SELECT SUM(year) FROM table_name_12 WHERE poin...,ensign n180b a Chassis with 4 points,True


In [30]:
answer = dataset['test'][0:10]['answer']

finetuned_model_results = rouge.compute(
    predictions=finetuned_model_answers,
    references=answer,
    use_aggregator=True,
    use_stemmer=True,
)

base_model_results = rouge.compute(
    predictions=base_model_answers,
    references=answer,
    use_aggregator=True,
    use_stemmer=True,
)

In [32]:
print('Fine-tuned\n',finetuned_model_results)
print('Base\n',base_model_results)

Fine-tuned
 {'rouge1': 0.47508313963883425, 'rouge2': 0.2219484050147666, 'rougeL': 0.42335379541887685, 'rougeLsum': 0.42434819317798045}
Base
 {'rouge1': 0.19241943734015343, 'rouge2': 0.07258823529411765, 'rougeL': 0.17581084825234441, 'rougeLsum': 0.17674381926683713}
